In [5]:
import streamlit as st
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go

df = pd.read_csv('/Users/devi/Documents/Practicum/simple-streamlit-datavis-app/vehicles_us.csv')
df['manufacturer'] = df['model'].apply(lambda x: x.split()[0])

In [15]:
display(df.info())

display(df.describe())

display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  manufacturer  51525 non-null  object 
dtypes: float64(4), int64(2), object(8)
memory usage: 5.5+ MB


None

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
0,10900,2016.0,nissan rogue,excellent,4.0,gas,0.0,automatic,wagon,NaN,1.0,2019-04-13,177,nissan
1,12900,2015.0,toyota camry,excellent,4.0,gas,0.0,automatic,sedan,NaN,NaN,2019-03-21,128,toyota
2,6900,2015.0,hyundai elantra,good,4.0,gas,0.0,automatic,sedan,NaN,NaN,2019-03-17,117,hyundai
3,5495,2000.0,chevrolet silverado 1500,good,NaN,gas,0.0,automatic,truck,NaN,1.0,2019-01-25,115,chevrolet
4,5995,2003.0,ram 2500,good,6.0,diesel,0.0,automatic,truck,NaN,NaN,2018-09-07,105,ram


In [27]:
df['model_year'] = df['model_year'].astype('int')

df['cylinders'] = df['cylinders'].astype('int')

# replace null values in the 'is_4wd' column with the value 0
df['is_4wd'].fillna(value=0, inplace=True)
df['is_4wd'] = df['is_4wd'].astype('int')


# compute the median of non-null values in the 'model_year' column for each model group
medians = df.groupby('model')['model_year'].transform(lambda x: x.median())
# replace null values in the 'model_year' column with the median of non-null values for each model group
df['model_year'].fillna(medians, inplace=True)


# compute the median of non-null values in the 'cylinders' column for each model group
medians = df.groupby('model')['cylinders'].transform(lambda x: x.median())
# replace null values in the 'cylinders' column with the median of non-null values for each model group
df['cylinders'].fillna(medians, inplace=True)


# compute the median of non-null values in the 'odometer' column for each model_year group
medians = df.groupby('model_year')['odometer'].transform(lambda x: x.median())
# replace null values in the 'odometer' column with the median of non-null values for each model+year group
df['odometer'].fillna(medians, inplace=True)


# replace null values in the 'paint_color' column with the value 'NA'
df['paint_color'].fillna(value='NA', inplace=True)

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  int64  
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51524 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  int64  
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  manufacturer  51525 non-null  object 
dtypes: float64(2), int64(4), object(8)
memory usage: 5.5+ MB


None

In [26]:
display(df[df['odometer'].isna()])

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
45176,18000,1929.0,ford f-150,good,8.0,gas,NaN,manual,other,silver,0,2018-11-18,59,ford


In [ ]:
st.header('Data viewer')
show_manuf_1k_ads = st.checkbox('Include manufacturers with less than 1000 ads')
if not show_manuf_1k_ads:
    df = df.groupby('manufacturer').filter(lambda x: len(x) > 1000)

st.dataframe(df)
st.header('Vehicle types by manufacturer')
st.write(px.histogram(df, x='manufacturer', color='type'))
st.header('Histogram of `condition` vs `model_year`')

# -------------------------------------------------------
# histograms in plotly:
# fig = go.Figure()
# fig.add_trace(go.Histogram(x=df[df['condition']=='good']['model_year'], name='good'))
# fig.add_trace(go.Histogram(x=df[df['condition']=='excellent']['model_year'], name='excellent'))
# fig.update_layout(barmode='stack')
# st.write(fig)
# works, but too many lines of code
# -------------------------------------------------------

# histograms in plotly_express:
st.write(px.histogram(df, x='model_year', color='condition'))
# a lot more concise!
# -------------------------------------------------------

st.header('Compare price distribution between manufacturers')
manufac_list = sorted(df['manufacturer'].unique())
manufacturer_1 = st.selectbox('Select manufacturer 1',
                              manufac_list, index=manufac_list.index('chevrolet'))

manufacturer_2 = st.selectbox('Select manufacturer 2',
                              manufac_list, index=manufac_list.index('hyundai'))
mask_filter = (df['manufacturer'] == manufacturer_1) | (df['manufacturer'] == manufacturer_2)
df_filtered = df[mask_filter]
normalize = st.checkbox('Normalize histogram', value=True)
if normalize:
    histnorm = 'percent'
else:
    histnorm = None
st.write(px.histogram(df_filtered,
                      x='price',
                      nbins=30,
                      color='manufacturer',
                      histnorm=histnorm,
                      barmode='overlay'))